# 유동인구 분석

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
%config InlineBackend.figure_format = 'retina'
!apt -qq -y install fonts-nanum # 폰트 설치

The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import font_manager as fm, rc
import matplotlib as mpl
%matplotlib inline
# 한글 처리
path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties( fname=path, size=10 )
rc( 'font', family=font.get_name() )
mpl.font_manager._rebuild()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
# 유동인구 데이터 불러오기
path = '/content/drive/My Drive/Phoenix/Dacon_Data/fpopl.csv'
fpop = pd.read_csv(path)
fpop.head(10)

,base_ymd,tmzon_se_code,sexdstn_se_code,agrde_se_code,adstrd_code,popltn_cascnt
0,20200101,0,M,age_00,11650560,15
1,20200101,0,M,age_00,11590620,6
2,20200101,0,M,age_00,11560710,4
3,20200101,0,M,age_00,11470680,12
4,20200101,0,M,age_00,11350665,6
5,20200101,0,M,age_00,11305575,10
6,20200101,0,M,age_00,11290725,12
7,20200101,0,F,age_00,11740540,12
8,20200101,0,F,age_00,11680600,16
9,20200101,0,F,age_00,11620525,9


In [6]:
fpop.shape

(47546443, 6)

In [7]:
fpop.dtypes

base_ymd            int64
tmzon_se_code       int64
sexdstn_se_code    object
agrde_se_code      object
adstrd_code         int64
popltn_cascnt       int64
dtype: object

In [8]:
#결측치확인
fpop.isnull().sum()

base_ymd           0
tmzon_se_code      0
sexdstn_se_code    0
agrde_se_code      0
adstrd_code        0
popltn_cascnt      0
dtype: int64

In [9]:
fpop.tmzon_se_code.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [10]:
fpop.sexdstn_se_code.unique()

array(['M', 'F'], dtype=object)

In [11]:
# 행정정보 데이터 
path = '/content/drive/My Drive/Phoenix/Dacon_Data/adstrd_master.csv'
adstrd_info = pd.read_csv(path, encoding='utf-8')
adstrd_info.head(10)

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,11110515.0,청운효자동,서울특별시,종로구
1,11110530.0,사직동,서울특별시,종로구
2,11110540.0,삼청동,서울특별시,종로구
3,11110550.0,부암동,서울특별시,종로구
4,11110560.0,평창동,서울특별시,종로구
5,11110570.0,무악동,서울특별시,종로구
6,11110580.0,교남동,서울특별시,종로구
7,11110600.0,가회동,서울특별시,종로구
8,11110615.0,종로1.2.3.4가동,서울특별시,종로구
9,11110630.0,종로5.6가동,서울특별시,종로구


In [12]:
adstrd_info.shape

(3493, 4)

In [13]:
adstrd_info.dtypes

adstrd_code    float64
adstrd_nm       object
brtc_nm         object
signgu_nm       object
dtype: object

In [14]:
# 행정정보 코드 int형으로 바꾸기
adstrd_info['adstrd_code']=adstrd_info['adstrd_code'].apply(lambda x:int(x))

In [15]:
adstrd_info.dtypes

adstrd_code     int64
adstrd_nm      object
brtc_nm        object
signgu_nm      object
dtype: object

In [16]:
adstrd_info.head(2)

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,11110515,청운효자동,서울특별시,종로구
1,11110530,사직동,서울특별시,종로구


In [17]:
# 행정코드 기준으로 합치기
fpop_seoul = pd.merge(fpop, adstrd_info, on='adstrd_code', how='left')

In [18]:
fpop_seoul.shape

(47546443, 9)

In [19]:
fpop_seoul.head()

,base_ymd,tmzon_se_code,sexdstn_se_code,agrde_se_code,adstrd_code,popltn_cascnt,adstrd_nm,brtc_nm,signgu_nm
0,20200101,0,M,age_00,11650560,15,반포1동,서울특별시,서초구
1,20200101,0,M,age_00,11590620,6,사당1동,서울특별시,동작구
2,20200101,0,M,age_00,11560710,4,대림2동,서울특별시,영등포구
3,20200101,0,M,age_00,11470680,12,신정7동,서울특별시,양천구
4,20200101,0,M,age_00,11350665,6,상계3.4동,서울특별시,노원구


In [20]:
fpop_seoul.brtc_nm.unique()

array(['서울특별시'], dtype=object)

In [21]:
fpop_seoul.dtypes

base_ymd            int64
tmzon_se_code       int64
sexdstn_se_code    object
agrde_se_code      object
adstrd_code         int64
popltn_cascnt       int64
adstrd_nm          object
brtc_nm            object
signgu_nm          object
dtype: object

In [22]:
#'base_ymd' 컬럼 str형식으로 바꾸기
fpop_seoul['base_ymd'] = fpop_seoul['base_ymd'].apply(lambda x:str(x))

In [23]:
fpop_seoul.dtypes

base_ymd           object
tmzon_se_code       int64
sexdstn_se_code    object
agrde_se_code      object
adstrd_code         int64
popltn_cascnt       int64
adstrd_nm          object
brtc_nm            object
signgu_nm          object
dtype: object

In [ ]:
fpop_seoul['base_ymd'].apply(lambda x:f'{x[:4]}-{x[4:6]}-{x[6:]}')
  

In [ ]:
# 구별 유동인구 합치기
tmp = pd.pivot_table(fpop_seoul, index =['base_ymd'],columns=['signgu_nm'],values=['popltn_cascnt'], aggfunc=[np.sum, np.mean, len])
tmp.head(10)

In [ ]:
tmp.index

In [ ]:
# plt.figure(figsize=(20,10))
# plt.plot(tmp.index,'sum',data=tmp, marker='s')
# plt.show()

In [ ]:
#관악구	구로구	금천구	노원구	도봉구	동작구	서대문구	서초구	양천구	영등포구	은평구